# 进程与多线程

## 创建子进程

In [1]:
from multiprocessing import Process
import os

def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Process(target=run_proc, args=('test',))
    print('Child process will start.')
    p.start()
    p.join()  # 等待子进程结束后再继续往下运行
    print('Child process end.')

Parent process 79544.
Child process will start.
Run child process test (79553)...
Child process end.


## 子进程的输入输出

In [2]:
import subprocess

subprocess.call(['nslookup', 'www.python.org'])

0

In [3]:
p = subprocess.Popen(['nslookup'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, err = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('utf-8'))
print('Exit code:', p.returncode)

# 相当于在命令行执行命令nslookup，然后手动输入：
# set q=mx
# python.org
# exit

Server:		64.104.123.245
Address:	64.104.123.245#53

Non-authoritative answer:
python.org	mail exchanger = 50 mail.python.org.

Authoritative answers can be found from:
org	nameserver = d0.org.afilias-nst.org.
org	nameserver = a2.org.afilias-nst.info.
org	nameserver = c0.org.afilias-nst.info.
org	nameserver = a0.org.afilias-nst.info.
org	nameserver = b2.org.afilias-nst.org.
org	nameserver = b0.org.afilias-nst.org.
a0.org.afilias-nst.info	internet address = 199.19.56.1
a2.org.afilias-nst.info	internet address = 199.249.112.1
b0.org.afilias-nst.org	internet address = 199.19.54.1
b2.org.afilias-nst.org	internet address = 199.249.120.1
c0.org.afilias-nst.info	internet address = 199.19.53.1
d0.org.afilias-nst.org	internet address = 199.19.57.1
a0.org.afilias-nst.info	has AAAA address 2001:500:e::1
a2.org.afilias-nst.info	has AAAA address 2001:500:40::1
b0.org.afilias-nst.org	has AAAA address 2001:500:c::1
b2.org.afilias-nst.org	has AAAA address 2001:500:48::1
c0.org.afilias-nst.info	has AAAA

## 进程间通信

父进程中创建两个子进程，一个往 Queue 里写数据，一个从 Queue 里读数据：

In [4]:
from multiprocessing import Process, Queue
import os, time, random

def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)
        
q = Queue()
pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
pw.start()
pr.start()
pw.join()
pr.terminate()  # pr 进程里是死循环，无法等待其结束，只能强行终止:

Process to write: 79556
Put A to queue...
Process to read: 79557
Get A from queue.
Put B to queue...
Get B from queue.
Put C to queue...
Get C from queue.


## 创建线程

In [5]:
import time, threading

def worker():
    print('thread %s is running...' % threading.current_thread().name)
    time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

t = threading.Thread(target=worker, name='WorkerThread')
t.start()
t.join()

thread WorkerThread is running...
thread WorkerThread ended.


## 给线程加锁

In [8]:
import threading
from concurrent.futures import ThreadPoolExecutor

lock = threading.Lock()
count = 0

def run_thread_without_lock():
    global count
    while count < 10:
        count += 1
        print("\x1b[31m%s\x1b[0m, " % count, end='')

def run_thread_with_lock():
    global count
    with lock:
        while count < 10:
            count += 1
            print("%s, " % count, end='')
        
with ThreadPoolExecutor(max_workers=4) as executor:
    for i in range(5):
        executor.submit(run_thread_without_lock)
        
count = 0

with ThreadPoolExecutor(max_workers=4) as executor:
    for i in range(5):
        executor.submit(run_thread_with_lock)

7, 1, 2, 1, 9, 3, 4, 2, 10, 3, 5, 4, 6, 5, 8, 6, 7, 8, 9, 10, 

## thread local 变量

一个线程使用自己的局部变量比使用全局变量好，因为局部变量只有线程自己能看见，不会影响其他线程，而全局变量的修改必须加锁。

最常用的地方就是为每个线程绑定一个数据库连接，HTTP 请求，用户身份信息等，这样一个线程的所有调用到的处理函数都可以非常方便地访问这些资源。

In [7]:
import threading, time

tl = threading.local()

def worker(name):
    tl.name = name
    time.sleep(3)
    print("in thread:", tl.name)
    
t = threading.Thread(target=worker, args=("hello",), name='Thread-A')
t.start()
tl.name = 'world'
print("in main:", tl.name)

in main: world
in thread: hello


## 线程池

In [3]:
from concurrent.futures import ThreadPoolExecutor
import urllib.request

def fetch_url(url):
    u = urllib.request.urlopen(url)
    data = u.read()
    return data

pool = ThreadPoolExecutor(10)
# Submit work to the pool
a = pool.submit(fetch_url, 'http://www.python.org')
b = pool.submit(fetch_url, 'http://www.pypy.org')

# Get the results back
x = a.result()
y = b.result()